**Questions**

Combien d’ID RNB sont partagés entre le bailleur et PrioRéno ?
Combien sont juste dans un des deux jeu de données ?
Est-ce que pour un ID RNB donné le bailleur social et PrioRéno LS parlent du même bâtiment physique.

In [1]:
import pandas as pd

bailleur_path = 'mosellis.xlsx'
prioreno_path = 'prioreno_mosellis.xlsx'

## Colonnes dispo


In [3]:
df_bailleur_raw = pd.read_excel(bailleur_path)
df_prioreno_raw = pd.read_excel(prioreno_path)

print("Colonnes - bailleur :")
print(df_bailleur_raw.columns.tolist())
print()

print("Colonnes - PrioRéno :")
print(df_prioreno_raw.columns.tolist())

Colonnes - bailleur :
['fid', 'cleabs', 'identifiants_rnb', 'ID-Moselis', 'Code programme', 'Code tranche', 'Code ensemble', "Type d'ensemble", 'Adresse', 'Ville', 'Collectif / Individuel', 'Destination', "Bassin d'habitat", 'EPCI']

Colonnes - PrioRéno :
['Département', 'Adresse principale', 'Commune', 'Code postal', "Nombre d'étages", 'Nombre de logements sociaux', 'Nombre total de logements', 'Année de construction', 'Identifiant RNB', 'Code ensemble immobilier', 'Latitude', 'Longitude', 'Surface des logements', 'Fiabilité rattachement (Open data/Données indiv. agrégées)', 'Consommation finale globale gaz (Open data/Données indiv. agrégées) (kWh)', 'Consommation finale globale électricité (Open data/Données indiv. agrégées) (kWh)', 'Consommation finale globale total (Open data/Données indiv. agrégées) (kWh)', 'Surface logement HLM électricité  (Open data/Données indiv. agrégées)', 'Fiabilité rattachement (API)', "Chauffage - Type d'équipement", 'Chauffage - Energie principale chauff

In [4]:
# === 1. Normalisation côté bailleur ===
df_bailleur = df_bailleur_raw.rename(columns={"identifiants_rnb": "ID_RNB"}).copy()

# ID en chaînes propres, sans espaces, et suppression des vides
df_bailleur["ID_RNB"] = (
    df_bailleur["ID_RNB"]
    .astype(str)
    .str.strip()
    .replace({"": pd.NA, "nan": pd.NA})
)
df_bailleur = df_bailleur.dropna(subset=["ID_RNB"])

print("Colonnes bailleur :", df_bailleur.columns.tolist())
print("Nb de lignes bailleur après nettoyage :", len(df_bailleur))

# === 2. Normalisation côté PrioRéno ===
df_prioreno = df_prioreno_raw.rename(columns={"Identifiant RNB": "ID_RNB"}).copy()

# Remplacer les NaN par chaîne vide, trim, puis split sur les espaces
df_prioreno["ID_RNB"] = (
    df_prioreno["ID_RNB"]
    .fillna("")          # NaN -> ""
    .astype(str)
    .str.strip()
)

# On transforme chaque cellule en liste d'IDs (split sur les espaces, y compris multiples)
df_prioreno["ID_RNB"] = df_prioreno["ID_RNB"].str.split()

# On "explose" pour avoir une ligne par ID_RNB
df_prioreno = df_prioreno.explode("ID_RNB")

# Nettoyage : remove vides éventuels
df_prioreno["ID_RNB"] = (
    df_prioreno["ID_RNB"]
    .astype(str)
    .str.strip()
    .replace({"": pd.NA, "nan": pd.NA})
)
df_prioreno = df_prioreno.dropna(subset=["ID_RNB"])

print(" ")
print("Colonnes PrioRéno :", df_prioreno.columns.tolist())
print("Nb de lignes PrioRéno après éclatement/clean :", len(df_prioreno))

Colonnes bailleur : ['fid', 'cleabs', 'ID_RNB', 'ID-Moselis', 'Code programme', 'Code tranche', 'Code ensemble', "Type d'ensemble", 'Adresse', 'Ville', 'Collectif / Individuel', 'Destination', "Bassin d'habitat", 'EPCI']
Nb de lignes bailleur après nettoyage : 801
 
Colonnes PrioRéno : ['Département', 'Adresse principale', 'Commune', 'Code postal', "Nombre d'étages", 'Nombre de logements sociaux', 'Nombre total de logements', 'Année de construction', 'ID_RNB', 'Code ensemble immobilier', 'Latitude', 'Longitude', 'Surface des logements', 'Fiabilité rattachement (Open data/Données indiv. agrégées)', 'Consommation finale globale gaz (Open data/Données indiv. agrégées) (kWh)', 'Consommation finale globale électricité (Open data/Données indiv. agrégées) (kWh)', 'Consommation finale globale total (Open data/Données indiv. agrégées) (kWh)', 'Surface logement HLM électricité  (Open data/Données indiv. agrégées)', 'Fiabilité rattachement (API)', "Chauffage - Type d'équipement", 'Chauffage - Ene

In [5]:
# Combien de lignes Mosellis contiennent un ID RNB également présent dans PrioRéno ?

# Ensemble des IDs PrioRéno
ids_prioreno = set(df_prioreno["ID_RNB"])

# Sélection des lignes bailleur dont l'ID existe dans PrioRéno
df_bailleur_matched = df_bailleur[df_bailleur["ID_RNB"].isin(ids_prioreno)]



print(f"Nombre de lignes Mosellis avec un ID RNB présent dans PrioRéno :", len(df_bailleur_matched))
print("Nombre de lignes dans Mosellis : ", len(df_bailleur))
print("Pourcentage : ", (len(df_bailleur_matched) / len(df_bailleur) * 100 ))

Nombre de lignes Mosellis avec un ID RNB présent dans PrioRéno : 494
Nombre de lignes dans Mosellis :  801
Pourcentage :  61.67290886392009


In [7]:
# Extrait des lignes qui matchent

df_joint = df_bailleur_matched.merge(
    df_prioreno,
    on="ID_RNB",
    how="left",
    suffixes=("_MOS", "_PR")
)

# Adresse LMH : ADRESSE IKOS RECTIFIE + VILLE
df_joint["ADRESSE_MOS"] = (
    df_joint["Adresse"].astype(str).str.strip()
)

# Adresse PrioRéno : Adresse principale + CP + Commune
df_joint["ADRESSE_PR"] = (
    df_joint["Adresse principale"].astype(str).str.strip()
)

lat = df_joint["Latitude"].astype(str).str.replace(",", ".", regex=False)
lng = df_joint["Longitude"].astype(str).str.replace(",", ".", regex=False)

# Concatenate as "lat,lng"
df_joint["lat_lng"] = lat + "," + lng

# =========================
# 4. Extrait des lignes qui matchent
# =========================
extrait = df_joint[[
    "ID_RNB",
    "ADRESSE_MOS",
    "ADRESSE_PR",
    "lat_lng"
]].sample(n=100, random_state=42)

print(extrait.to_string(index=False))

      ID_RNB                                        ADRESSE_MOS                                            ADRESSE_PR                           lat_lng
MS5MCYPBJ4VW                  14 RUE GEORGES DUCROCQ 57070 METZ                     10 RUE GEORGES DUCROCQ 57070 METZ 49.1078933662404,6.18749587530735
W9PF84K5CBJW                      3 LES GERANIUMS 57600 FORBACH                           0001 LES GERANIUMS  FORBACH 49.1775889617142,6.88668975614402
GQRPJVWHAWF1               50 BOUCLE DU BREUIL 57100 THIONVILLE                  56 BOUCLE DU BREUIL 57100 THIONVILLE 49.3668714178312,6.10821344248647
VBS8EMZEC73K                       2 RUE DES PINS 57770 MOUSSEY                      1 RUE DES TILLEULS 57770 MOUSSEY 48.6881575677475,6.80217553639129
12HCKQEPPJRV                 4 RUE PAUL ALBERT 57100 THIONVILLE                    4 RUE PAUL ALBERT 57100 THIONVILLE 49.3564540554193,6.15627972117682
CX18Y1MJ2MK2                1 RUE DU CHEVREUIL 57100 THIONVILLE                   7 RUE 